# Outlook API Credential Guide

Extract Outlook Emails via Microsoft Graph API

1. Register Your App on Azure Portal

   Go to Azure Portal > Azure Active Directory > App registrations

   Click “New registration”

   Choose Mobole/Desktop app
   
   After creation, note the Application (client) ID

2. API Permissions

   Go to API permissions tab

   Click Add permission > Microsoft Graph > Delegated

   Choose: Mail.Read

   Click Grant admin consent

3. Allow public client flows

   Navigate to: Azure Active Directory > App registrations > Your App

   Go to Authentication tab

   Under "Advanced settings" → "Allow public client flows", set to "Yes"

   Save changes

In [ ]:
!pip install msal requests

In [ ]:
from msal import PublicClientApplication
import os
from dotenv import load_dotenv
import requests

In [ ]:
load_dotenv()

CLIENT_ID = os.getenv("AZURE_CLIENT_ID")
AUTHORITY = "https://login.microsoftonline.com/common" 
SCOPES = ["Mail.Read"]

app = PublicClientApplication(CLIENT_ID, authority=AUTHORITY)

flow = app.initiate_device_flow(scopes=SCOPES)
print("Go to:", flow["verification_uri"])
print("Enter code:", flow["user_code"])

result = app.acquire_token_by_device_flow(flow)

if "access_token" not in result:
    raise Exception("Failed to authenticate:", result)

access_token = result["access_token"]

In [ ]:
print("Granted scopes:", result.get("scope"))

headers = {
    "Authorization": f"Bearer {access_token}",
    "Prefer": "outlook.body-content-type='text'"
}

query = (
    "https://graph.microsoft.com/v1.0/me/messages"
    "?$top=1"
    "&$select=id,subject,receivedDateTime,body"
)

all_emails = []

while query:
    response = requests.get(query, headers=headers)

    if not response.ok:
        print(response.text)
        print(f"❌ HTTP {response.status_code}: {response.text}")
        break

    try:
        res = response.json()
    except ValueError:
        print("❌ Invalid JSON:", response.text)
        break

    for msg in res.get("value", []):
        all_emails.append({
            "id": msg.get("id"),
            "subject": msg.get("subject", ""),
            "body": msg.get("body", {}).get("content", ""),
            "date": msg.get("receivedDateTime", "")
        })

    query = res.get("@odata.nextLink")


In [ ]:
all_emails